In [ ]:
!pip uninstall -y cupy-cuda11x cupy-cuda12x cupy

In [ ]:
!pip install cupy-cuda12x

In [ ]:
!apt-cache search cuda-cusparse

In [ ]:
!nvidia-smi

In [ ]:
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
from scipy.optimize import curve_fit

# --- Map and derivative using GPU ---
def f_SB(x, b):
    return cp.where(x <= 1 / b, b * x, (b * x - 1) / (b - 1))

def df_SB(x, b):
    return cp.where(x <= 1 / b, b, b / (b - 1))

# --- Evolve system on GPU ---
def evolve_system_gpu(N, p, T, epsilon, b, seed):
    rng = np.random.default_rng(seed)

    G = nx.erdos_renyi_graph(N, p, seed=rng, directed=False)
    A_cpu = nx.adjacency_matrix(G).astype(np.float32)
    A_cpu.setdiag(1.0)
    A_cpu = A_cpu.tocsr()

    deg = np.array(A_cpu.sum(axis=1)).flatten()
    deg_safe = np.where(deg == 0, 1, deg)
    A_gpu = cp.sparse.csr_matrix(A_cpu)
    deg_safe_gpu = cp.asarray(deg_safe)

    y = cp.asarray(rng.uniform(0, 1, N), dtype=cp.float32)
    v = cp.asarray(rng.uniform(-1e-8, 1e-8, N), dtype=cp.float32)

    le_sum = 0.0
    for _ in range(T):
        local = f_SB(y, b)
        coupling = A_gpu @ local / deg_safe_gpu
        y = (1 - epsilon) * local + epsilon * coupling

        J = df_SB(y, b)
        v = (1 - epsilon) * J * v + epsilon * (A_gpu @ (J * v) / deg_safe_gpu)

        v_norm = cp.linalg.norm(v)
        if v_norm == 0:
            continue
        le_sum += cp.log(cp.abs(v_norm))
        v /= v_norm

    return float(le_sum / T)

# --- λ(N) for a single p ---
def compute_LE_for_single_p(N_list, T, epsilon, b, p, n_trials=4):
    lambda_list = []
    for N in N_list:
        print(f"\n>>> Running N = {N} <<<")
        seeds = np.arange(n_trials) + 100
        results = [evolve_system_gpu(N, p, T, epsilon, b, seed) for seed in seeds]
        lambda_avg = np.mean(results)
        lambda_list.append(lambda_avg)
    return np.array(lambda_list)

# --- R² utility ---
def r2_score(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return 1 - ss_res / ss_tot

# --- Main simulation, plotting, saving ---
def run_simulation_single_p(N_list, p, T=1000, epsilon=0.3, b=4, n_trials=4, filename="lyap_data.npz"):
    # Compute λ(N)
    lambda_array = compute_LE_for_single_p(N_list, T, epsilon, b, p, n_trials)

    # λ(N) vs 1/ln(N)
    plt.figure(figsize=(12, 5))
    x1 = 1 / np.log(N_list)
    coeffs1 = np.polyfit(x1, lambda_array, 1)
    y1_fit = np.polyval(coeffs1, x1)
    r2_1 = r2_score(lambda_array, y1_fit)

    plt.subplot(1, 2, 1)
    plt.plot(x1, lambda_array, 'bo', label='Data')
    plt.plot(x1, y1_fit, 'g--', label='Fit')
    plt.xlabel('1 / ln(N)')
    plt.ylabel('λ(N)')
    plt.title(f'Logarithmic Fit\n$R^2$ = {r2_1:.4f}')
    plt.legend()

    # ln(λ∞ - λ(N)) vs ln(N)
    lambda_inf = np.mean(lambda_array[-3:])
    delta = np.abs(lambda_inf - lambda_array) + 1e-12
    x2 = np.log(N_list)
    y2 = np.log(delta)
    coeffs2 = np.polyfit(x2, y2, 1)
    y2_fit = np.polyval(coeffs2, x2)
    r2_2 = r2_score(y2, y2_fit)
    gamma = -coeffs2[0]

    plt.subplot(1, 2, 2)
    plt.plot(x2, y2, 'bo', label='Data')
    plt.plot(x2, y2_fit, 'g--', label='Fit')
    plt.xlabel('ln(N)')
    plt.ylabel('ln(λ∞ - λ(N))')
    plt.title(f'Power-law Fit, γ ≈ {gamma:.2f}\n$R^2$ = {r2_2:.4f}')
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Save data
    np.savez(filename, N_list=N_list, p=p, lambda_array=lambda_array,
             lambda_inf=lambda_inf, gamma=gamma, r2_log=r2_1, r2_power=r2_2)
    print(f"Data saved to {filename}")

    return lambda_array, gamma, r2_1, r2_2



In [ ]:
N_list = [400, 800, 1600, 3200, 6400, 12800, 25600]
p = 0.2
lambda_array, gamma, r2_log, r2_power = run_simulation_single_p(N_list, p)